In [ ]:
import pandas as pd
import numpy as np
import math

from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/CS3237
!ls

Mounted at /content/drive
/content/drive/My Drive/CS3237
'Data Processing_Fall.ipynb'   fall_57_processed.csv
'Data Processing_LSTM.ipynb'   fall_58_processed.csv
 down_up_processed.csv	       fall_59_processed.csv
 fall_100_processed.csv        fall_5_processed.csv
 fall_10_processed.csv	       fall_60_processed.csv
 fall_11_processed.csv	       fall_61_processed.csv
 fall_12_processed.csv	       fall_62_processed.csv
 fall_13_processed.csv	       fall_63_processed.csv
 fall_14_processed.csv	       fall_64_processed.csv
 fall_15_processed.csv	       fall_65_processed.csv
 fall_16_processed.csv	       fall_66_processed.csv
 fall_17_processed.csv	       fall_67_processed.csv
 fall_18_processed.csv	       fall_68_processed.csv
 fall_19_processed.csv	       fall_69_processed.csv
 fall_1_processed.csv	       fall_6_processed.csv
 fall_20_processed.csv	       fall_70_processed.csv
 fall_21_processed.csv	       fall_71_processed.csv
 fall_22_processed.csv	       fall_72_processed.csv
 fall_2

In [ ]:
dataframe = pd.read_csv('moving_turnright_processed.csv') # train on moving_turnleft_processed.csv, moving_turnright_processed.csv separately
dataframe = dataframe.drop(columns=dataframe.columns[0], axis=1)
print(dataframe)

       accX  accY  accZ  accResultant   gyrX   gyrY   gyrZ  isFall
0     -0.03  0.04  1.08          1.09 -14.96  -2.91 -13.61     1.0
1     -0.02  0.03  1.10          1.10 -21.84  -7.23 -10.17     1.0
2     -0.02  0.03  1.10          1.10 -23.29  -9.78  -6.98     1.0
3     -0.03  0.02  1.10          1.11 -18.69 -12.22  -0.40     1.0
4     -0.13 -0.01  1.12          1.13 -12.14 -14.72   4.77     1.0
...     ...   ...   ...           ...    ...    ...    ...     ...
10923  0.90  0.10  0.01          0.90  -0.07   0.08  -0.06     1.0
10924  0.90  0.10  0.01          0.90  -0.05   0.11  -0.08     1.0
10925  0.90  0.10  0.01          0.90  -0.04   0.08  -0.07     1.0
10926  0.90  0.10  0.00          0.90  -0.10   0.08  -0.06     1.0
10927  0.90  0.10  0.01          0.90  -0.06   0.12  -0.06     1.0

[10928 rows x 8 columns]


In [ ]:
from sklearn.preprocessing import MinMaxScaler

dataset = dataframe.to_numpy()
X = dataset[:, :-1]
Y = dataset[:, -1:]

scaler = MinMaxScaler(feature_range = (0, 1))
# dataset = scaler.fit_transform(dataset)
X = scaler.fit_transform(X)

# print(dataset)
print(X)
print(Y)

[[0.21604938 0.5952381  0.78756477 ... 0.56615171 0.23616987 0.29578568]
 [0.22222222 0.58333333 0.79792746 ... 0.54226697 0.21303486 0.32273226]
 [0.22222222 0.58333333 0.79792746 ... 0.53723312 0.19937878 0.34772051]
 ...
 [0.79012346 0.66666667 0.23316062 ... 0.61794827 0.2521823  0.40184866]
 [0.79012346 0.66666667 0.22797927 ... 0.61773998 0.2521823  0.40192699]
 [0.79012346 0.66666667 0.23316062 ... 0.61787884 0.25239651 0.40192699]]
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [ ]:
MODEL_NAME = 'lstm.hd5'
savemodel = ModelCheckpoint(MODEL_NAME)

In [ ]:
"""
Adjust these to control # of epochs, etc.
"""
batch_size = 1
look_back = 10
skip = 1
hidden_size = 128
num_epochs = 15
TRAIN_PERCENT = 0.7

# Set the random number seed so that we have reproducible results
# numpy.random.seed(7)

In [ ]:
from keras.utils import to_categorical

# Figure out how many vectors for training and how many for testing.

train_size = int(len(dataset) * TRAIN_PERCENT)
test_size = len(dataset) - train_size

# Slice the dataset accordingly
train, test  = dataset[:train_size], dataset[train_size:]

# Generate the datasets with the given look_back.
def create_dataset(dataset, look_back=look_back):
    dataX, dataY= [], []

    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i+look_back), 0:-1].flatten()
        dataX.append(a)
        dataY.append(dataset[i + look_back, -1])
    return np.array(dataX), np.array(dataY)


In [ ]:
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
import os

def build_model(model_name):
    if os.path.exists(model_name):
        model = load_model(model_name)
    else:
        model = Sequential()
        model.add(LSTM(128, batch_input_shape=(batch_size, look_back*7, 1), stateful = True))
        model.add(Dense(8, activation='softmax'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
model = build_model(MODEL_NAME)

num_epochs = 150
for i in range(num_epochs):
    print("Iter: %d of %d" % (i+1, num_epochs))
    model.fit(trainX, trainY, epochs=1, batch_size = batch_size, shuffle = False, 
               validation_data=(testX, testY), callbacks=[savemodel])
    model.reset_states()


Iter: 1 of 150
7636/7638 [============================>.] - ETA: 0s - loss: 4.8295 - accuracy: 0.5572

7638/7638 [==============================] - 68s 8ms/step - loss: 4.8284 - accuracy: 0.5573 - val_loss: 2.1536 - val_accuracy: 0.5946
Iter: 2 of 150
7637/7638 [============================>.] - ETA: 0s - loss: 2.4576 - accuracy: 0.5682

7638/7638 [==============================] - 60s 8ms/step - loss: 2.4573 - accuracy: 0.5682 - val_loss: 1.4982 - val_accuracy: 0.6059
Iter: 3 of 150
7631/7638 [============================>.] - ETA: 0s - loss: 1.6494 - accuracy: 0.6073

7638/7638 [==============================] - 69s 9ms/step - loss: 1.6481 - accuracy: 0.6076 - val_loss: 1.6751 - val_accuracy: 0.5793
Iter: 4 of 150
7630/7638 [============================>.] - ETA: 0s - loss: 1.2854 - accuracy: 0.6014

7638/7638 [==============================] - 70s 9ms/step - loss: 1.2844 - accuracy: 0.6016 - val_loss: 0.9955 - val_accuracy: 0.6255
Iter: 5 of 150
7631/7638 [============================>.] - ETA: 0s - loss: 0.9581 - accuracy: 0.6091

...

7638/7638 [==============================] - 60s 8ms/step - loss: 0.6497 - accuracy: 0.6730 - val_loss: 0.6903 - val_accuracy: 0.6833
Iter: 147 of 150
7638/7638 [==============================] - ETA: 0s - loss: 0.6439 - accuracy: 0.6739

7638/7638 [==============================] - 60s 8ms/step - loss: 0.6439 - accuracy: 0.6739 - val_loss: 0.6826 - val_accuracy: 0.6680
Iter: 148 of 150
7631/7638 [============================>.] - ETA: 0s - loss: 0.6221 - accuracy: 0.6868

7638/7638 [==============================] - 69s 9ms/step - loss: 0.6218 - accuracy: 0.6871 - val_loss: 0.6901 - val_accuracy: 0.6812
Iter: 149 of 150
7630/7638 [============================>.] - ETA: 0s - loss: 0.6059 - accuracy: 0.6946

7638/7638 [==============================] - 60s 8ms/step - loss: 0.6056 - accuracy: 0.6949 - val_loss: 0.6567 - val_accuracy: 0.6821
Iter: 150 of 150
7630/7638 [============================>.] - ETA: 0s - loss: 0.5931 - accuracy: 0.6996

7638/7638 [==============================] - 62s 8ms/step - loss: 0.5928 - accuracy: 0.6998 - val_loss: 0.7505 - val_accuracy: 0.6628
